In [31]:
%pip install pandas sqlalchemy pymysql mysql-connector-python PyMySQL


Note: you may need to restart the kernel to use updated packages.


In [41]:
from sqlalchemy import create_engine, text
import pandas as pd
from getpass import getpass

In [42]:
# import mysql password
pwd = getpass()

In [43]:
# Replace these with your actual credentials:
username = "root"
password = (pwd)
host     = "localhost"
port     = 3306
database = "sakila"

In [44]:
con_url = f"mysql+pymysql://{username}:{password}@{host}/{database}"
engine = create_engine(con_url)

In [59]:


def rentals_month(engine, month, year):
    query = f"""
        SELECT customer_id, rental_date
        FROM rental
        WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    df = pd.read_sql(query, engine)
    return df


In [62]:
rentals_aug = rentals_month(engine, 8, 2005)
rentals_aug

,customer_id,rental_date
0,113,2005-08-01 00:00:44
1,578,2005-08-01 00:08:01
2,377,2005-08-01 00:08:01
3,77,2005-08-01 00:09:33
4,64,2005-08-01 00:12:11
...,...,...
5681,14,2005-08-23 22:25:26
5682,74,2005-08-23 22:26:47
5683,114,2005-08-23 22:42:48
5684,103,2005-08-23 22:43:07


In [63]:
def rental_count_month(df, month, year):
    name = f"rentals_{str(month).zfill(2)}_{year}"
    result = df.groupby('customer_id').size().reset_index(name=name)
    return result


In [66]:
counts = rental_count_month(rentals_aug, 8, 2005)
print(counts.head())


   customer_id  rentals_08_2005
0            1               11
1            2               11
2            3                7
3            4               11
4            5               13


In [69]:
def compare_rentals(df1, df2):
    combined = pd.merge(df1, df2, on='customer_id')

    col1 = combined.columns[1]
    col2 = combined.columns[1]

    combined['difference']= combined[col2] - combined[col1]

    return combined


In [72]:
may_rentals = rentals_month(engine, 5, 2005)
may_rentals


,customer_id,rental_date
0,130,2005-05-24 22:53:30
1,459,2005-05-24 22:54:33
2,408,2005-05-24 23:03:39
3,333,2005-05-24 23:04:41
4,222,2005-05-24 23:05:21
...,...,...
1151,506,2005-05-31 21:36:44
1152,59,2005-05-31 21:42:09
1153,251,2005-05-31 22:17:11
1154,106,2005-05-31 22:37:34


In [73]:
may_count = rental_count_month(may_rentals, 5, 2005)
may_count

,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


In [74]:
june_rentals = rentals_month(engine, 6, 2005)
june_count = rental_count_month(june_rentals, 6, 2005)

comparison = compare_rentals(may_count, june_count)
print(comparison.head())


   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1                2                7           0
1            2                1                1           0
2            3                2                4           0
3            5                3                5           0
4            6                3                4           0
